In [ ]:
import sqlite3
from sqlite3 import Error
import pandas as pd
import numpy as np
import csv
import requests
import os

#Muda o diretório de trabalho
print(os.getcwd())
workdir_path = 'D:\PainelTelefonia\ETL'
os.chdir(workdir_path)
print(os.getcwd())#Muda o diretório de trabalho
print(os.getcwd())
workdir_path = 'D:\PainelTelefonia\ETL'
os.chdir(workdir_path)
print(os.getcwd())

###  Ramais IP

#### Identifica os dispositivos VoIP

In [ ]:
# Consulta ao banco SQLite
databaseCL1 = 'cluster01_2021_09_05.db3'
try:
    conn1 = sqlite3.connect(databaseCL1)
except Error as e:
    print(e)
registered = pd.read_sql_query("SELECT IP,DN,UserAgent,LastRegister FROM EndPoints", conn1)
registered = dfCL1.groupby(['UserAgent','DN']).tail(1)
conn1.close()

#### Identifica os tipos de dispositivos

In [ ]:
#Categoriza por dispositivo
registered['DISPOSITIVO'] = pd.np.where(registered.UserAgent.str.contains("Aparelho IP Tipo I"), "Aparelho IP Tipo I",
                                pd.np.where(registered.UserAgent.str.contains("Aparelho IP Tipo II"), "Aparelho IP Tipo II",
                                pd.np.where(registered.UserAgent.str.contains("Aparelho IP Tipo III"), "Aparelho IP Tipo III",
                                pd.np.where(registered.UserAgent.str.contains("Softphone Tipo I"), "Softphone Tipo I",
                                pd.np.where(registered.UserAgent.str.contains("Softphone Tipo II"), "Softphone Tipo II",
                                pd.np.where(registered.UserAgent.str.contains("Aparelho Analogico"), "Aparelho Analogico",
                                pd.np.where(registered.UserAgent.str.contains("Aparelho Digital"), "Aparelho Digital", "Outros")))))))

registered = registered.groupby(['DISPOSITIVO','DN']).tail(1)

#### Limpeza dos dados 

In [ ]:
#Elimina linhas com dados nulos
registered.dropna(inplace=True)
#Elimina linhas com valores diferentes de um IP no campo IP
registered = registered[(registered['IP'].str.len() <= 15.0)&(registered['IP'].str.startswith('1'))].copy()

#### Identifica a estação

In [ ]:
import requests

registered['ESTACAO']=""

#consulta uma aplicação que fornece a localidade com base no IP informado
for index, row in registered.iterrows():
    
    print(index)
    ip = row['IP']
    response = requests.get('http://127.0.0.1:5000/getsite/' + str(ip))
    site = response.text.replace('"','').rstrip()
    registered.loc[index,'ESTACAO'] = site

#### Ramais IP distintos

In [ ]:
# Agragação da quantidade de ramais IP distintos por estação
ramaisIPdistintos = registered.groupby(['DN']).tail(1)
ramais_IP_distintos_agreg = ramaisIPdistintos.groupby(['ESTACAO'])['DN'].nunique().reset_index(name='QUANTIDADE')

#### Dispositivos IP distintos

In [ ]:
# Agregação da quantidade de dispositivos IP distintos por estação
ramais_ip_agreg = registered.groupby(['ESTACAO','DISPOSITIVO'])['DN'].nunique().reset_index(name='QUANTIDADE')

### Ramais TDM

In [ ]:
#Coleta dados dos ramais TDM do banco Oracle
import cx_Oracle
dsn_tns = cx_Oracle.makedsn('servidor', '1000', service_name='tele') 
connection = cx_Oracle.connect(user='usuario', password='password', dsn=dsn_tns) # cria conexao
cursor = connection.cursor() # cria um cursor

query = """SELECT
               RAMAL,
               DISPOSITIVO,
               ESTACAO,
           FROM basetdm"""

ramais_tdm = pd.read_sql(query, con=connection)

cursor.close()
connection.close()

In [ ]:
#Agrega por DISPOSITIVO e conta a quantidade de ramais
ramais_tdm_agreg = ramais_tdm.groupby(['ESTACAO','DISPOSITIVO'])['RAMAL'].nunique().reset_index(name='QUANTIDADE')

### Criação das tabelas do staging

#### Dispositivos Distintos

In [ ]:
#Concatena os os dados agregados de dispositivos IP e TDM
dispositivos_distintos_agreg = pd.concat([ramais_ip_agreg,ramais_tdm_agreg])

#Cria coluna com a respectiva semana
dispositivos_distintos_agreg['SEMANA']="2021/06/01"

#Cria CSV com dados de dispositivos distintos
dispositivos_distintos_agreg.to_csv('DispositivosDistintos-2021_06.csv',index=False)

ramais_agreg.head()

#### Ramais Distintos

In [ ]:
#Concatena os dados agregados de ramais IP e TDM
IP = ramais_IP_distintos_agreg.copy()
IP.rename(columns={"QUANTIDADE": "QUANT_IP"}, inplace=True)

TDM = ramais_tdm_agreg.groupby(['ESTACAO']).sum().reset_index()
TDM.rename(columns={"QUANTIDADE": "QUANT_TDM"}, inplace=True)

ramais_distintos_agreg = pd.merge(IP, TDM, on='ESTACAO', how='outer').fillna(0)
ramais_distintos_agreg = ramais_distintos_agreg.astype({'QUANT_IP': 'int32', 'QUANT_TDM' : 'int32'})

#Cria coluna com a respectiva semana
ramais_distintos_agreg['SEMANA'] = '2021/06/01'

#Cria CSV com dados de ramais distintos
ramais_distintos_agreg.to_csv('RamaisDistintos-2021_06.csv',index=False)

ramais_distintos_agreg.head()